In [1]:
import cv2
import numpy as np
from gtts import gTTS
import os
from pydub import AudioSegment
from pydub.playback import play

from keras.models import load_model
model = load_model('./model_save')

c:\Python311\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
#speech engine
def speak(text):
    tts=gTTS(text=text, lang='en')
    tts.save("output.mp3")
    os.system("start kmplayer.exe output.mp3")

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)
    
    x1 = int(0.7*frame.shape[1])
    y1 = 100
    x2 = frame.shape[1]-10
    y2 = int(0.45*frame.shape[1])
    # Drawing the ROI
    # The increment/decrement by 1 is to compensate for the bounding box
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    # Extracting the ROI
    roi = frame[y1:y2, x1:x2]
    roi = cv2.resize(roi, (100, 100)) 
    
    
    #_, mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)
    #kernel = np.ones((1, 1), np.uint8)
    #img = cv2.dilate(mask, kernel, iterations=1)
    #img = cv2.erode(mask, kernel, iterations=1)
    # do the processing after capturing the image!
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    #_, roi = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    roi = cv2.Canny(roi,80,80)
    
    roi1 = roi.copy()
    cv2.imshow("ROI", roi)
    
    
    roi1.resize(1,100,100,1)
    #classIndex = int(model.predict(roi1))
    predicted_classes=model.predict(roi1)
    classIndex=np.argmax(predicted_classes)
    m = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'nothing',15:'O',16:'P',17:'Q',18:'R',19:'S',20:'T',21:'U',22:'V',23:'W',24:'X',25:'Y',26:'Z'}
    
    alphabet=m[classIndex]
    cv2.putText(frame, "Alphabet : "+str(m[classIndex]), (10, 430), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 2)
    
    cv2.imshow("Frame", frame)
    speak(alphabet)
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 21ms/step
